In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!python /content/drive/MyDrive/cv_project_fall_2022/utils/resource_downloader.py -r mscoco_dataset

In [3]:
import sys
sys.path.append('/content/drive/MyDrive/cv_project_fall_2022/')
#sys.path.append('/content/drive/MyDrive/cv_project_fall_2022/models/definitions')

In [4]:
import os
import argparse
import time

import torch
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter
import numpy as np

from models.definitions.perceptual_loss_net import PerceptualLossNet
from models.definitions.transformer_net_new import TransformerNet
import utils.utils as utils

In [5]:
import os
from os.path import abspath, expanduser
from typing import Any, Callable, Dict, List, Optional, Tuple, Union
import matplotlib.pyplot as plt

import torch
from PIL import Image

from torchvision.datasets.utils  import download_and_extract_archive, download_file_from_google_drive, extract_archive, verify_str_arg
from torchvision.datasets import VisionDataset

verbose = True

class WIDERFace(VisionDataset):
    """`WIDERFace <http://shuoyang1213.me/WIDERFACE/>`_ Dataset.

    Args:
        root (string): Root directory where images and annotations are downloaded to.
            Expects the following folder structure if download=False:

            .. code::

                <root>
                    └── widerface
                        ├── wider_face_split ('wider_face_split.zip' if compressed)
                        ├── WIDER_train ('WIDER_train.zip' if compressed)
                        ├── WIDER_val ('WIDER_val.zip' if compressed)
                        └── WIDER_test ('WIDER_test.zip' if compressed)
        split (string): The dataset split to use. One of {``train``, ``val``, ``test``}.
            Defaults to ``train``.
        transform (callable, optional): A function/transform that  takes in a PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.

    """

    BASE_FOLDER = "widerface"
    FILE_LIST = [
        # File ID                             MD5 Hash                            Filename
        ("15hGDLhsx8bLgLcIRD5DhYt5iBxnjNF1M", "3fedf70df600953d25982bcd13d91ba2", "WIDER_train.zip"),
        ("1GUCogbp16PMGa39thoMMeWxp7Rp5oM8Q", "dfa7d7e790efa35df3788964cf0bbaea", "WIDER_val.zip"),
        #("1HIfDbVEWKmsYKJZm4lchTBDLW5N7dY5T", "e5d8f4248ed24c334bbd12f49c29dd40", "WIDER_test.zip"),
    ]
    ANNOTATIONS_FILE = (
        "http://shuoyang1213.me/WIDERFACE/support/bbx_annotation/wider_face_split.zip",
        "0e3767bcf0e326556d407bf5bff5d27c",
        "wider_face_split.zip",
    )

    def __init__(
        self,
        root: str,
        split: str = "train",
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        download: bool = False,
    ) -> None:
        super().__init__(
            root=os.path.join(root, self.BASE_FOLDER), transform=transform, target_transform=target_transform
        )
        # check arguments
        self.split = verify_str_arg(split, "split", ("train", "val", "test"))

        if download:
            self.download()

        if not self._check_integrity():
            raise RuntimeError("Dataset not found or corrupted. You can use download=True to download and prepare it")

        self.img_info: List[Dict[str, Union[str, Dict[str, torch.Tensor]]]] = []
        if self.split in ("train", "val"):
            self.parse_train_val_annotations_file()
        else:
            self.parse_test_annotations_file()

        print("Dataset Size : ", len(self.img_info))

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is a dict of annotations for all faces in the image.
            target=None for the test split.
        """

        # stay consistent with other datasets and return a PIL Image
        img = Image.open(self.img_info[index]["img_path"], mode='r')
        
        #img.show()
        #print(img.shape)
        if self.transform is not None:
            img = self.transform(img)
        #print(img.shape)
        #plt.imshow(img[0])
        mask_path = self.img_info[index]["mask_path"]
        

        if verbose :
            print("Image Path : ", self.img_info[index]["img_path"])
            print("Mask Path : ", mask_path)
        
        face_mask = None
        if(os.path.exists(mask_path)):
            face_mask = Image.open(mask_path)
            #face_mask = Image.open(mask_path, mode='r')
        
        target = None if self.split == "test" else face_mask

        

        # face_masked_image_path = self.img_info[index]["face_masked_image_path"]
        # face_masked_image = None
        # if(os.path.exists(face_masked_image_path)):
        #     face_masked_image = Image.open(face_masked_image_path, mode='r')
        
        # target = None if self.split == "test" else face_masked_image

        # if verbose :
            # print("face_masked_image_path Path : ", face_masked_image_path)
            # print("Mask Path : ", mask_path)
            # print("Type(Image) : ", type(img))
            # print("Type(target) : ", type(target))

        
        #print(target.shape)
        #plt.imshow(target[0])
        #if target is None:

        #TODO : Remove this - its a temp fix
        #target = torch.tensor([0])
        if self.target_transform is not None and target is not None:
            target = self.target_transform(target)
        
        return img, target


    def __len__(self) -> int:
        return len(self.img_info)

    def extra_repr(self) -> str:
        lines = ["Split: {split}"]
        return "\n".join(lines).format(**self.__dict__)

    def parse_train_val_annotations_file(self) -> None:
        filename = "wider_face_train_bbx_gt.txt" if self.split == "train" else "wider_face_val_bbx_gt.txt"
        #filepath = "/content/drive/MyDrive/cv_project_fall_2022/face_mask/data/wider_face_split/wider_face_train_bbx_gt.txt"
        filepath = os.path.join(self.root, "wider_face_split", filename)

        with open(filepath) as f:
            lines = f.readlines()
            file_name_line, num_boxes_line, box_annotation_line = True, False, False
            num_boxes, box_counter = 0, 0
            labels = []
            for line in lines:
                line = line.rstrip()
                if file_name_line:
                    img_path = os.path.join(self.root, "WIDER_" + self.split, "images", line)
                    mask_path = os.path.join(self.root, "WIDER_" + self.split, "masks", line)
                    overlay_mask_path = os.path.join(self.root, "WIDER_" + self.split, "overlay_mask", line)
                    face_masked_image_path = os.path.join(self.root, "WIDER_" + self.split, "face_masked", line)
            
                    img_path = abspath(expanduser(img_path))
                    file_name_line = False
                    num_boxes_line = True
                elif num_boxes_line:
                    num_boxes = int(line)
                    num_boxes_line = False
                    box_annotation_line = True
                elif box_annotation_line:
                    box_counter += 1
                    line_split = line.split(" ")
                    line_values = [int(x) for x in line_split]
                    labels.append(line_values)
                    face_count_threshold = 5
                    if box_counter >= num_boxes:
                        box_annotation_line = False
                        file_name_line = True
                        labels_tensor = torch.tensor(labels)
                        if num_boxes < face_count_threshold:
                            self.img_info.append(
                                {
                                    "img_path": img_path,
                                    "mask_path" : mask_path,
                                    "overlay_mask_path" : overlay_mask_path,
                                    "face_masked_image_path" : face_masked_image_path,
                                    "annotations": {
                                        "bbox": labels_tensor[:, 0:4],  # x, y, width, height
                                        "blur": labels_tensor[:, 4],
                                        "expression": labels_tensor[:, 5],
                                        "illumination": labels_tensor[:, 6],
                                        "occlusion": labels_tensor[:, 7],
                                        "pose": labels_tensor[:, 8],
                                        "invalid": labels_tensor[:, 9],
                                    },
                                }
                            )
                        box_counter = 0
                        labels.clear()
                else:
                    raise RuntimeError(f"Error parsing annotation file {filepath}")

    def parse_test_annotations_file(self) -> None:
        filepath = os.path.join(self.root, "wider_face_split", "wider_face_test_filelist.txt")
        filepath = abspath(expanduser(filepath))
        with open(filepath) as f:
            lines = f.readlines()
            for line in lines:
                line = line.rstrip()
                img_path = os.path.join(self.root, "WIDER_test", "images", line)
                img_path = abspath(expanduser(img_path))
                self.img_info.append({"img_path": img_path})

    def _check_integrity(self) -> bool:
        # Allow original archive to be deleted (zip). Only need the extracted images
        all_files = self.FILE_LIST.copy()
        all_files.append(self.ANNOTATIONS_FILE)
        for (_, md5, filename) in all_files:
            file, ext = os.path.splitext(filename)
            extracted_dir = os.path.join(self.root, file)
            if not os.path.exists(extracted_dir):
                print("extracted_dir : ", extracted_dir)
                return False
        return True

    def download(self) -> None:
        if self._check_integrity():
            print("Files already downloaded and verified")
            return

        # download and extract image data
        for (file_id, md5, filename) in self.FILE_LIST:
            download_file_from_google_drive(file_id, self.root, filename, md5)
            filepath = os.path.join(self.root, filename)
            extract_archive(filepath)

        # download and extract annotation files
        download_and_extract_archive(
            url=self.ANNOTATIONS_FILE[0], download_root=self.root, md5=self.ANNOTATIONS_FILE[1]
        )


In [ ]:
# Test and Visualize edge model


# RCF Model
from models.definitions.hed_net_new import RCF

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
hed_model = RCF()
hed_model.to(device)
hed_model.eval()
hed_model_bin_path = "/content/drive/MyDrive/cv_project_fall_2022/models/binaries/bsds500_pascal_model.pth"
checkpoint = torch.load(hed_model_bin_path)
hed_model.load_state_dict(checkpoint)
# hed_state = torch.load(hed_model_bin_path)['state_dict']
# hed_state.keys()
# hed_model.load_state_dict(hed_state, strict=False)
for param in hed_model.parameters():
    param.requires_grad = False

In [ ]:

#
# Modifiable args - feel free to play with these (only a small subset is exposed by design to avoid cluttering)
#
default_config = dict()
parser = argparse.ArgumentParser()
# training related
#parser.add_argument("--style_img_name", type=str, help="style image name that will be used for training", default='edtaonisl.jpg')
default_config['style_img_name'] = 'mosaic.jpg'

#parser.add_argument("--content_weight", type=float, help="weight factor for content loss", default=1e0)  # you don't need to change this one just play with style loss
default_config['content_weight'] = 1e0
#parser.add_argument("--style_weight", type=float, help="weight factor for style loss", default=4e5)
default_config['style_weight'] = 4e5
#parser.add_argument("--tv_weight", type=float, help="weight factor for total variation loss", default=0)
default_config['tv_weight'] = 0
default_config['structure_weight'] = 1

default_config['depth_loss_weight'] = 5
default_config['face_edge_loss_weight'] = 1e5


#parser.add_argument("--num_of_epochs", type=int, help="number of training epochs ", default=2)
default_config['num_of_epochs'] = 10
#parser.add_argument("--subset_size", type=int, help="number of MS COCO images (NOT BATCHES) to use, default is all (~83k)(specified by None)", default=None)
default_config['subset_size'] = 100
# logging/debugging/checkpoint related (helps a lot with experimentation)
#parser.add_argument("--enable_tensorboard", type=bool, help="enable tensorboard logging (scalars + images)", default=True)
default_config['enable_tensorboard'] = False
#parser.add_argument("--image_log_freq", type=int, help="tensorboard image logging (batch) frequency - enable_tensorboard must be True to use", default=100)
#default_config['image_log_freq'] = 100
#parser.add_argument("--console_log_freq", type=int, help="logging to output console (batch) frequency", default=500)
default_config['console_log_freq'] = 10
#parser.add_argument("--checkpoint_freq", type=int, help="checkpoint model saving (batch) frequency", default=2000)
default_config['checkpoint_freq'] = 100
#args = parser.parse_args()



In [ ]:
#
# Fixed args - don't change these unless you have a good reason
#
root = "/content/drive/MyDrive/cv_project_fall_2022"
dataset_path = os.path.join(root, 'data', 'mscoco') #,'train2017')
style_images_path = os.path.join(root, 'data', 'style-images')
model_binaries_path = os.path.join(root, 'models', 'binaries')
checkpoints_root_path = os.path.join(root, 'models', 'checkpoints')
image_size = 256  # all dataset images are resized to (image_size x image_size)
batch_size = 4

assert os.path.exists(dataset_path), f'MS COCO missing. Download the dataset using resource_downloader.py script.'
os.makedirs(model_binaries_path, exist_ok=True)


In [ ]:
checkpoints_path = os.path.join(checkpoints_root_path,default_config['style_img_name'].split('.')[0])
if default_config['checkpoint_freq'] is not None:
    os.makedirs(checkpoints_path, exist_ok=True)

# Wrapping training configuration into a dictionary
training_config = dict()
for arg in default_config.keys():
    training_config[arg] = default_config[arg]
training_config['dataset_path'] = dataset_path
training_config['style_images_path'] = style_images_path
training_config['model_binaries_path'] = model_binaries_path
training_config['checkpoints_path'] = checkpoints_path
training_config['image_size'] = image_size
training_config['batch_size'] = batch_size

# Original J.Johnson's training with improved transformer net architecture




In [ ]:
print(dataset_path)

/content/drive/MyDrive/cv_project_fall_2022/data/mscoco


In [ ]:
from torchvision import transforms
import numpy as np
from torch.utils.data import DataLoader

# training_config = {}
# training_config['image_size'] = 500


WIDERFACE_MEAN_1 = np.array([0.485, 0.456, 0.406])
WIDERFACE_MEAN_255 = np.array([123.675, 116.28, 103.53])

# Usually when normalizing 0..255 images only mean-normalization is performed -> that's why standard dev is all 1s here
WIDERFACE_STD_1 = np.array([0.229, 0.224, 0.225])
WIDERFACE_STD_NEUTRAL = np.array([1, 1, 1])



# transform_list = [transforms.Resize(training_config['image_size']),
#                   transforms.CenterCrop(training_config['image_size']),
#                   transforms.ToTensor()]

# if is_255_range:
#     transform_list.append(transforms.Lambda(lambda x: x.mul(255)))
# if should_normalize:
#     transform_list.append(transforms.Normalize(mean=WIDERFACE_MEAN_255, std=WIDERFACE_STD_NEUTRAL) if is_255_range else transforms.Normalize(mean=WIDERFACE_MEAN_1, std=WIDERFACE_STD_1))
# transform = transforms.Compose(transform_list)


# target_transform = transforms.Compose([
#     transforms.Resize(training_config['image_size']),
#     transforms.CenterCrop(training_config['image_size']),
#     transforms.ToTensor()
# ])


dataset_root = "/content/drive/MyDrive/cv_project_fall_2022/face_mask/data/"

def get_training_data_loader(training_config, should_normalize=False, is_255_range=False):
    """
        There are multiple ways to make this feed-forward NST working,
        including using 0..255 range (without any normalization) images during transformer net training,
        keeping the options if somebody wants to play and get better results.
    """
    transform_list = [transforms.Resize(training_config['image_size']),
                      transforms.CenterCrop(training_config['image_size']),
                      transforms.ToTensor()]
    target_transform_list = [transforms.Resize(training_config['image_size']),
                      transforms.CenterCrop(training_config['image_size']),
                      transforms.Grayscale(),
                      transforms.ToTensor()]
    
    if is_255_range:
        transform_list.append(transforms.Lambda(lambda x: x.mul(255)))
    if should_normalize:
        transform_list.append(transforms.Normalize(mean=WIDERFACE_MEAN_255, std=WIDERFACE_STD_NEUTRAL) if is_255_range else transforms.Normalize(mean=WIDERFACE_MEAN_1, std=WIDERFACE_STD_1))
    
    transform = transforms.Compose(transform_list)
    #transform_list.append(transforms.Grayscale())
    target_transform = transforms.Compose(target_transform_list)

    
    dataset_train = WIDERFace(dataset_root, split="train", download=False, transform=transform,target_transform=target_transform)


    # train_dataset = datasets.ImageFolder(training_config['dataset_path'], transform)
    # sampler = SequentialSubsetSampler(train_dataset, training_config['subset_size'])
    # training_config['subset_size'] = len(sampler)  # update in case it was None
    
    train_dataloader = DataLoader(dataset_train, batch_size=training_config['batch_size'], shuffle=True)
    print(f'Using {len(train_dataloader)*training_config["batch_size"]*training_config["num_of_epochs"]} datapoints ({len(train_dataloader)*training_config["num_of_epochs"]} batches) (MS COCO images) for transformer network training.')
    return train_dataloader

In [ ]:
train_loader = get_training_data_loader(training_config)

Dataset Size :  7935
Using 79360 datapoints (19840 batches) (MS COCO images) for transformer network training.


In [ ]:
# train_features, train_labels = next(iter(train_loader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")

In [ ]:
training_config

{'style_img_name': 'mosaic.jpg',
 'content_weight': 1.0,
 'style_weight': 400000.0,
 'tv_weight': 0,
 'structure_weight': 1,
 'depth_loss_weight': 5,
 'face_edge_loss_weight': 100000.0,
 'num_of_epochs': 10,
 'subset_size': 100,
 'enable_tensorboard': False,
 'console_log_freq': 10,
 'checkpoint_freq': 100,
 'dataset_path': '/content/drive/MyDrive/cv_project_fall_2022/data/mscoco',
 'style_images_path': '/content/drive/MyDrive/cv_project_fall_2022/data/style-images',
 'model_binaries_path': '/content/drive/MyDrive/cv_project_fall_2022/models/binaries',
 'checkpoints_path': '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/mosaic',
 'image_size': 256,
 'batch_size': 4}

In [ ]:
# # Midas Model
# !pip install timm
# #model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
# #model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
# model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)
# midas = torch.hub.load("intel-isl/MiDaS", model_type)


# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# midas = midas.eval().to(device)

# # midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

# # if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
# #     midas_transforms = midas_transforms.dpt_transform
# # else:
# #     midas_transforms = midas_transforms.small_transform

# for param in midas.parameters():
#     param.requires_grad = False


In [ ]:
# HED Model

# from models.definitions.hed_net import HED

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# hed_model = HED()
# hed_model.to(device)
# hed_model.eval()
# hed_model_bin_path = "/content/drive/MyDrive/cv_project_fall_2022/models/binaries/Hedcheckpoint_epoch9.pth"
# hed_state = torch.load(hed_model_bin_path)['state_dict']
# hed_state.keys()
# hed_model.load_state_dict(hed_state, strict=False)
# for param in hed_model.parameters():
#     param.requires_grad = False


# HED Model New
from models.definitions.hed_net_new import RCF

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
hed_model = RCF()
hed_model.to(device)
hed_model.eval()
hed_model_bin_path = "/content/drive/MyDrive/cv_project_fall_2022/models/binaries/bsds500_pascal_model.pth"
checkpoint = torch.load(hed_model_bin_path)
hed_model.load_state_dict(checkpoint)
# hed_state = torch.load(hed_model_bin_path)['state_dict']
# hed_state.keys()
# hed_model.load_state_dict(hed_state, strict=False)
for param in hed_model.parameters():
    param.requires_grad = False

In [ ]:
verbose = False
def train(training_config, load_checkpoint = False):
    writer = SummaryWriter()  # (tensorboard) writer will output to ./runs/ directory by default
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # prepare data loader
    train_loader = get_training_data_loader(training_config)

    # prepare neural networks
    transformer_net = TransformerNet().train().to(device)

    if(load_checkpoint):
        #path = "/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/edtaonisl/ckpt_style_edtaonisl_cw_1.0_sw_400000.0_tw_0_epoch_1_batch_1899.pth"
        #path = "/content/drive/MyDrive/cv_project_fall_2022/models/binaries/edtaonisl_9e5_33k.pth"
        path = "/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/mosaic/ckpt_style_depth_face_edge_layer_mosaic_cw_1.0_sw_400000.0_tw_0_epoch_0_batch_499.pth"
        training_state = torch.load(path)
        state_dict = training_state["state_dict"]
        transformer_net.load_state_dict(state_dict, strict=True)
        print("Loaded !")

    perceptual_loss_net = PerceptualLossNet(requires_grad=False).to(device)

    optimizer = Adam(transformer_net.parameters())

    # Calculate style image's Gram matrices (style representation)
    # Built over feature maps as produced by the perceptual net - VGG16
    style_img_path = os.path.join(training_config['style_images_path'], training_config['style_img_name'])
    style_img = utils.prepare_img(style_img_path, target_shape=None, device=device, batch_size=training_config['batch_size'])
    style_img_set_of_feature_maps = perceptual_loss_net(style_img)
    target_style_representation = [utils.gram_matrix(x) for x in style_img_set_of_feature_maps]

    style_img_for_face = utils.prepare_img(style_img_path, target_shape=(training_config['image_size'],training_config['image_size']), device=device, batch_size=training_config['batch_size'])
    style_img_face_set_of_feature_maps = perceptual_loss_net(style_img_for_face)

    utils.print_header(training_config)
    # Tracking loss metrics, NST is ill-posed we can only track loss and visual appearance of the stylized images
    acc_content_loss, acc_style_loss, acc_tv_loss, acc_content_face_loss, acc_style_face_loss, acc_depth_loss, acc_face_edge_loss = [0., 0., 0., 0., 0., 0., 0.]
    ts = time.time()
    for epoch in range(training_config['num_of_epochs']):
        # epoch = 2
        for batch_id, (content_batch, face_masked_image_batch) in enumerate(train_loader):


            
            if(batch_id == 1980):
                break

            if verbose :
                print("Content Batch : ")
                print(type(content_batch))
                print(content_batch.shape)
                print(type(face_masked_image_batch))
                print(face_masked_image_batch.shape)
                plt.imshow(content_batch[0].T)
                plt.show()
                plt.imshow(face_masked_image_batch[0].squeeze().T)
                plt.show()
            
            content_batch = content_batch.to(device)
            face_masked_image_batch = face_masked_image_batch.to(device)
            
            result = torch.cat([content_batch, face_masked_image_batch], dim=1)
            result = result.to(device)
            stylized_batch = transformer_net(result)
            
            # plt.imshow(content_batch.cpu().detach().numpy()[0].T)
            # plt.show()

            # Midas
            #input_batch = midas_transforms(content_batch).to(device)
            
            
            
            # print(content_batch.shape)
            # print(content_depth_image.shape)
            # print(styled_depth_image.shape)

            # plt.imshow(content_depth_image.cpu().detach().numpy()[0].T)
            # plt.show()

            # plt.imshow(styled_depth_image.cpu().detach().numpy()[0].T)
            # plt.show()
            
            
            
            # HED
            # plt.imshow(content_batch.cpu().detach().numpy()[0].T)
            # plt.show()

            
            
            # content_hed_result_cpu = content_hed_result.cpu().detach().numpy()
            # plt.imshow(content_hed_result_cpu[0].squeeze().T)
            # plt.show()
            
            # styled_hed_result_cpu = styled_hed_result.cpu().detach().numpy()
            # plt.imshow(styled_hed_result_cpu[0].squeeze().T)
            # plt.show()
            
            ''' Depth Loss '''
            # content_depth_image = midas(content_batch)
            # styled_depth_image = midas(stylized_batch)
            # depth_loss  = training_config['structure_weight'] * training_config['depth_loss_weight'] * torch.nn.MSELoss(reduction='mean')(content_depth_image, styled_depth_image)
            depth_loss = torch.tensor([0]).to(device)

            
            ''' Face Edge Loss '''
            # First mask and then edge detect
            content_face_masked = torch.mul(content_batch, face_masked_image_batch)
            styled_face_masked = torch.mul(stylized_batch, face_masked_image_batch)

            content_hed_face_masked = hed_model(content_face_masked)[0]
            styled_hed_face_masked = hed_model(styled_face_masked)[0]

            # print(content_hed_result.shape)
            # print(styled_hed_result.shape)

            # print("Max Content :", torch.max(content_hed_result[0]) )
            # print("Max HED.    :", torch.max(styled_hed_result[0]) )
            # print("Max Mask    :", torch.max(face_masked_image_batch[0]) )


            # print("Count Non Zero Content :", torch.count_nonzero(content_hed_result[0]) )
            # print("Count Non Zero HED.    :", torch.count_nonzero(styled_hed_result[0]) )
            # print("Count Non Zero Mask    :", torch.count_nonzero(face_masked_image_batch[0]) )

            # print(face_masked_image_batch[0])
            # print(content_hed_result[0])
            # print(styled_hed_result[0])

            # content_hed_result_cpu = content_hed_result.cpu().detach().numpy()
            # plt.imshow(content_hed_result_cpu[0].squeeze().T)
            # plt.show()
            
            # styled_hed_result_cpu = styled_hed_result.cpu().detach().numpy()
            # plt.imshow(styled_hed_result_cpu[0].squeeze().T)
            # plt.show()

            
            # plt.imshow((styled_hed_result_cpu[0] - content_hed_result_cpu[0]).squeeze().T)
            # plt.show()

            # break
            # First detect and then mask
            # content_hed_result = hed_model(content_batch)[0]
            # styled_hed_result = hed_model(stylized_batch)[0]

            # # print("Max Content :", torch.max(content_hed_result[0]) )
            # # print("Max HED.    :", torch.max(styled_hed_result[0]) )
            
            # content_hed_face_masked = torch.mul(content_hed_result, face_masked_image_batch)
            # styled_hed_face_masked = torch.mul(styled_hed_result, face_masked_image_batch)


            # print(content_face_masked.shape)
            # print(styled_face_masked.shape)

            # print("Max Content :", torch.max(content_face_masked[0]) )
            # print("Max HED.    :", torch.max(styled_face_masked[0]) )
            # print("Max Mask    :", torch.max(face_masked_image_batch[0]) )


            # print("Count Non Zero Content :", torch.count_nonzero(content_face_masked[0]) )
            # print("Count Non Zero HED.    :", torch.count_nonzero(styled_face_masked[0]) )
            # print("Count Non Zero Mask    :", torch.count_nonzero(face_masked_image_batch[0]) )

            # print(content_face_masked[0])
            # print(styled_face_masked[0])

            # idx_content_hed = torch.nonzero(content_face_masked[0])
            # print(content_face_masked[0][idx_content_hed])

            # idx_style_hed = torch.nonzero(styled_face_masked[0])
            # print(styled_face_masked[0][idx_style_hed])

            # content_hed_result_cpu = content_face_masked.cpu().detach().numpy()
            # plt.imshow(content_hed_result_cpu[0].squeeze().T)
            # plt.show()
            
            # styled_hed_result_cpu = styled_face_masked.cpu().detach().numpy()
            # plt.imshow(styled_hed_result_cpu[0].squeeze().T)
            # plt.show()

            # face_masked_cpu = face_masked_image_batch.cpu().detach().numpy()
            # plt.imshow(face_masked_cpu[0].squeeze().T)
            # plt.show()
            # break

            # results_content_face_masked = torch.mul(content_hed_result, face_masked_image_batch)[0]
            # results_styled_face_masked = torch.mul(styled_hed_result, face_masked_image_batch)[0]

            # max_c = torch.max(results_content_face_masked)
            # max_s = torch.max(results_styled_face_masked)
            # print("----------------------------------------")
            # print("Max Content :", max_c)
            # print("Max HED.    :", max_s)
            
            # print(torch.numel(results_content_face_masked[results_content_face_masked > 0 ]))
            # print(torch.numel(results_content_face_masked[results_content_face_masked < 1 ]))
            # print(torch.numel(results_content_face_masked[results_content_face_masked < (max_c.item() - 0.4) ]))

            # print(torch.numel(results_styled_face_masked[ results_styled_face_masked > 0]))
            # print(torch.numel(results_styled_face_masked[ results_styled_face_masked < 1]))
            # print(torch.numel(results_styled_face_masked[ results_styled_face_masked < (max_s.item() - 0.4)]))

            # content_hed_result_cpu = results_content_face_masked.cpu().detach().numpy() * 5
            # plt.imshow(content_hed_result_cpu.squeeze().T,cmap='gray', vmin=0, vmax=1)
            # plt.show()
            
            # styled_hed_result_cpu = results_styled_face_masked.cpu().detach().numpy() * 5
            # plt.imshow(styled_hed_result_cpu.squeeze().T,cmap='gray', vmin=0, vmax=1)
            # plt.show()

            # break

            # content_hed_result = hed_model(content_batch)
            # styled_hed_result = hed_model(stylized_batch)

            # print("Max Content :", torch.max(content_hed_result[0]) )
            # print("Max HED.    :", torch.max(styled_hed_result[0]) )
            
            # content_face_masked = torch.mul(content_hed_result, face_masked_image_batch)
            # styled_face_masked = torch.mul(styled_hed_result, face_masked_image_batch)


            # print(content_face_masked.shape)
            # print(styled_face_masked.shape)

            # print("Max Content :", torch.max(content_face_masked[0]) )
            # print("Max HED.    :", torch.max(styled_face_masked[0]) )
            # print("Max Mask    :", torch.max(face_masked_image_batch[0]) )


            # print("Count Non Zero Content :", torch.count_nonzero(content_face_masked[0]) )
            # print("Count Non Zero HED.    :", torch.count_nonzero(styled_face_masked[0]) )
            # print("Count Non Zero Mask    :", torch.count_nonzero(face_masked_image_batch[0]) )

            # print(content_face_masked[0])
            # print(styled_face_masked[0])

            # idx_content_hed = torch.nonzero(content_face_masked[0])
            # print(content_face_masked[0][idx_content_hed])

            # idx_style_hed = torch.nonzero(styled_face_masked[0])
            # print(styled_face_masked[0][idx_style_hed])

            # content_hed_result_cpu = content_face_masked.cpu().detach().numpy()
            # plt.imshow(content_hed_result_cpu[0].squeeze().T)
            # plt.show()
            
            # styled_hed_result_cpu = styled_face_masked.cpu().detach().numpy()
            # plt.imshow(styled_hed_result_cpu[0].squeeze().T)
            # plt.show()

            # face_masked_cpu = face_masked_image_batch.cpu().detach().numpy()
            # plt.imshow(face_masked_cpu[0].squeeze().T)
            # plt.show()

            # break


            face_edge_loss  = training_config['structure_weight'] * training_config['face_edge_loss_weight'] * torch.nn.MSELoss(reduction='mean')(content_hed_face_masked, styled_hed_face_masked)


            if verbose :
                print("Masked Batch :",face_masked_image_batch.shape)
                print("Non Zero in mask :", torch.count_nonzero(face_masked_image_batch))
                print("Non Zero in masked batch :", torch.count_nonzero(face_masked_image_batch))
            
            ''' Image Style and Content Losses '''
            content_batch_set_of_feature_maps = perceptual_loss_net(content_batch)
            stylized_batch_set_of_feature_maps = perceptual_loss_net(stylized_batch)



            ''' Content Loss - Full Image '''
            target_content_representation = content_batch_set_of_feature_maps.relu2_2
            current_content_representation = stylized_batch_set_of_feature_maps.relu2_2
            content_loss = training_config['content_weight'] * torch.nn.MSELoss(reduction='mean')(target_content_representation, current_content_representation)
            


            ''' Style Loss - Full Image '''
            style_loss = 0.0
            current_style_representation = [utils.gram_matrix(x) for x in stylized_batch_set_of_feature_maps]
            
            for gram_gt, gram_hat in zip(target_style_representation, current_style_representation):
                style_loss += torch.nn.MSELoss(reduction='mean')(gram_gt, gram_hat)
            style_loss /= len(target_style_representation)
            style_loss *= training_config['style_weight']

            
            '''
            Face Losses
                1. Content Face Loss
                    a). Create masked features from output
                    b). Create masked features from content image
                    c). Compute loss
                
                2. Style Face Loss
                    a). Create masked style features from output
                    b). Make gram matrix from 1
                    c). Create masked style features from content image
                    d). Make gram matrix from 2
                    e). Compute loss
            '''


            ''' Content Face Loss '''
            content_loss_face =  torch.tensor([0]).to(device)
            # stylized_face_masked_batch = torch.mul(stylized_batch, face_masked_image_batch)
            # content_face_masked_image_batch = torch.mul(content_batch, face_masked_image_batch)
            # content_loss_face = training_config['content_weight'] * torch.nn.MSELoss(reduction='mean')(content_face_masked_image_batch, stylized_face_masked_batch)

            ''' Style Face Loss '''
            style_loss_face = torch.tensor([0]).to(device)
            
            # face_style_loss_layer = 2
            # filter_size = 2 ** face_style_loss_layer
            
            # style_filter_mask = torch.nn.functional.avg_pool2d(face_masked_image_batch, filter_size, stride=filter_size, padding=0)
            # masked_features_content = torch.mul(style_img_face_set_of_feature_maps[face_style_loss_layer], style_filter_mask)
            # content_face_style_representation = utils.gram_matrix(masked_features_content)

            # using style of masked content image
            # masked_features_content = torch.mul(content_batch_set_of_feature_maps[face_style_loss_layer], style_filter_mask)
            # content_face_style_representation = utils.gram_matrix(masked_features_content)
            
            # masked_features_styled = torch.mul(stylized_batch_set_of_feature_maps[face_style_loss_layer], style_filter_mask)
            # styled_face_style_representation = utils.gram_matrix(masked_features_styled)
        
            # style_loss_face = torch.nn.MSELoss(reduction='mean')(styled_face_style_representation, content_face_style_representation)
            # style_loss_face *= training_config['style_weight']
            
            
            # step5: Calculate total variation loss - enforces image smoothness
            tv_loss = training_config['tv_weight'] * utils.total_variation(stylized_batch)
            # losses = [content_loss , style_loss , tv_loss , content_loss_face , style_loss_face , depth_loss , face_edge_loss]
            # for i,x in enumerate(losses):
            #     print(i , " = ", x.get_device())
            # step6: Combine losses and do a backprop
            total_loss = content_loss + style_loss + tv_loss + content_loss_face + style_loss_face + depth_loss + face_edge_loss
            total_loss.backward()
            optimizer.step()

            # clear gradients for the next round
            optimizer.zero_grad()

            # Logging and checkpoint creation
            acc_content_loss += content_loss.item()
            acc_content_face_loss += content_loss_face.item()
            acc_style_loss += style_loss.item()
            acc_style_face_loss += style_loss_face.item()
            acc_tv_loss += tv_loss.item()
            acc_depth_loss += depth_loss.item()
            acc_face_edge_loss += face_edge_loss.item()

            if training_config['enable_tensorboard']:
                # log scalars
                writer.add_scalar('Loss/content-loss', content_loss.item(), len(train_loader) * epoch + batch_id + 1)
                writer.add_scalar('Loss/content-loss-face', content_loss_face.item(), len(train_loader) * epoch + batch_id + 1)
                writer.add_scalar('Loss/style-loss', style_loss.item(), len(train_loader) * epoch + batch_id + 1)
                writer.add_scalar('Loss/style-loss-face', style_loss_face.item(), len(train_loader) * epoch + batch_id + 1)
                writer.add_scalar('Loss/tv-loss', tv_loss.item(), len(train_loader) * epoch + batch_id + 1)
                writer.add_scalars('Statistics/min-max-mean-median', {'min': torch.min(stylized_batch), 'max': torch.max(stylized_batch), 'mean': torch.mean(stylized_batch), 'median': torch.median(stylized_batch)}, len(train_loader) * epoch + batch_id + 1)
                # log stylized image
                if batch_id % training_config['image_log_freq'] == 0:
                    stylized = utils.post_process_image(stylized_batch[0].detach().to('cpu').numpy())
                    stylized = np.moveaxis(stylized, 2, 0)  # writer expects channel first image
                    writer.add_image('stylized_img', stylized, len(train_loader) * epoch + batch_id + 1)

            if training_config['console_log_freq'] is not None and batch_id % training_config['console_log_freq'] == 0:
                #f'time elapsed={(time.time()-ts)/60:.2f}[min]|epoch={epoch + 1}'
                print(f'time elapsed={(time.time()-ts)/60:.2f}[min]|epoch={epoch + 1}' +
                                                                 f'|batch=[{batch_id + 1}/{len(train_loader)}]' +
                                                                 f'|c-loss={acc_content_loss / training_config["console_log_freq"]}' +
                                                                 f'|cf-loss={acc_content_face_loss / training_config["console_log_freq"]}' +
                                                                 f'|s-loss={acc_style_loss / training_config["console_log_freq"]}' +
                                                                 f'|sf-loss={acc_style_face_loss / training_config["console_log_freq"]}' +
                                                                 f'|tv-loss={acc_tv_loss / training_config["console_log_freq"]}' +
                                                                 f'|d-loss={acc_depth_loss / training_config["console_log_freq"]}' +
                                                                 f'|fe-loss={acc_face_edge_loss / training_config["console_log_freq"]}' +
                                                                 f'|total loss={(acc_content_loss + acc_style_loss + acc_style_face_loss + acc_tv_loss + acc_content_face_loss) / training_config["console_log_freq"]}')
                ts = time.time()
                #acc_content_loss, acc_style_loss, acc_tv_loss, acc_content_face_loss, acc_style_face_loss = [0., 0., 0., 0., 0.]
                acc_content_loss, acc_style_loss, acc_tv_loss, acc_content_face_loss, acc_style_face_loss, acc_depth_loss, acc_face_edge_loss = [0., 0., 0., 0., 0., 0., 0.]

            if training_config['checkpoint_freq'] is not None and (batch_id + 1) % training_config['checkpoint_freq'] == 0:
                training_state = utils.get_training_metadata(training_config)
                training_state["state_dict"] = transformer_net.state_dict()
                training_state["optimizer_state"] = optimizer.state_dict()
                ckpt_model_name = f"ckpt_style_face_edge_only_pre_mask_new_{training_config['style_img_name'].split('.')[0]}_cw_{str(training_config['content_weight'])}_sw_{str(training_config['style_weight'])}_tw_{str(training_config['tv_weight'])}_epoch_{epoch}_batch_{batch_id}.pth"
                print("Saving checkpoint : ", os.path.join(training_config['checkpoints_path'], ckpt_model_name))
                torch.save(training_state, os.path.join(training_config['checkpoints_path'], ckpt_model_name))

    # Save model with additional metadata - like which commit was used to train the model, style/content weights, etc.
    training_state = utils.get_training_metadata(training_config)
    training_state["state_dict"] = transformer_net.state_dict()
    training_state["optimizer_state"] = optimizer.state_dict()
    model_name = f"style_{training_config['style_img_name'].split('.')[0]}_datapoints_{training_state['num_of_datapoints']}_cw_{str(training_config['content_weight'])}_sw_{str(training_config['style_weight'])}_tw_{str(training_config['tv_weight'])}.pth"
    torch.save(training_state, os.path.join(training_config['model_binaries_path'], model_name))


In [ ]:
# /content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/mosaic/ckpt_style_face_layer_2_mosaic_cw_1.0_sw_400000.0_tw_0_epoch_3_batch_1899.pth
train(training_config, load_checkpoint = False)

Dataset Size :  7935
Using 79360 datapoints (19840 batches) (MS COCO images) for transformer network training.


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


Learning the style of mosaic.jpg style image.
********************************************************************************
Hyperparams: content_weight=1.0, style_weight=400000.0 and tv_weight=0
********************************************************************************
Logging to console every 10 batches.
Saving checkpoint models every 100 batches.
Tensorboard disabled.
********************************************************************************
time elapsed=0.40[min]|epoch=1|batch=[1/1984]|c-loss=0.6181422233581543|cf-loss=0.0|s-loss=18.561773681640624|sf-loss=0.0|tv-loss=0.0|d-loss=0.0|fe-loss=0.03293311297893524|total loss=19.17991590499878
time elapsed=2.70[min]|epoch=1|batch=[11/1984]|c-loss=4.189179396629333|cf-loss=0.0|s-loss=212.42249755859376|sf-loss=0.0|tv-loss=0.0|d-loss=0.0|fe-loss=1.1700837537646294|total loss=216.6116769552231
time elapsed=1.84[min]|epoch=1|batch=[21/1984]|c-loss=6.317735195159912|cf-loss=0.0|s-loss=161.8796813964844|sf-loss=0.0|tv-loss=0.0|d

In [ ]:
# Image_Path = "/content/drive/MyDrive/cv_project_fall_2022/face_mask/data/widerface/WIDER_train/images/9--Press_Conference/9_Press_Conference_Press_Conference_9_919.jpg"
# img = Image.open(Image_Path)


In [ ]:
# should_normalize=True
# is_255_range=False

# transform_list = [transforms.Resize(training_config['image_size']),
#                       transforms.CenterCrop(training_config['image_size']),
#                       transforms.ToTensor()]
# if is_255_range:
#     transform_list.append(transforms.Lambda(lambda x: x.mul(255)))
# if should_normalize:
#     transform_list.append(transforms.Normalize(mean=WIDERFACE_MEAN_255, std=WIDERFACE_STD_NEUTRAL) if is_255_range else transforms.Normalize(mean=WIDERFACE_MEAN_1, std=WIDERFACE_STD_1))

# transform = transforms.Compose(transform_list)
    
 

In [ ]:
# time elapsed=0.12[min]|epoch=1|batch=[1/3220]|c-loss=1.0365273475646972|s-loss=3.672317123413086|tv-loss=0.0|total loss=4.708844470977783
# time elapsed=1.74[min]|epoch=1|batch=[11/3220]|c-loss=6.942651319503784|s-loss=81.89519786834717|tv-loss=0.0|total loss=88.83784918785095
# time elapsed=2.96[min]|epoch=1|batch=[21/3220]|c-loss=7.069084644317627|s-loss=27.299999618530272|tv-loss=0.0|total loss=34.3690842628479
# time elapsed=4.24[min]|epoch=1|batch=[31/3220]|c-loss=7.0678380012512205|s-loss=22.420995903015136|tv-loss=0.0|total loss=29.488833904266357
# time elapsed=5.40[min]|epoch=1|batch=[41/3220]|c-loss=7.002507114410401|s-loss=21.557346534729003|tv-loss=0.0|total loss=28.559853649139406
# time elapsed=6.60[min]|epoch=1|batch=[51/3220]|c-loss=6.810258674621582|s-loss=20.481855392456055|tv-loss=0.0|total loss=27.292114067077637
# time elapsed=7.82[min]|epoch=1|batch=[61/3220]|c-loss=7.716735172271728|s-loss=18.954132652282716|tv-loss=0.0|total loss=26.670867824554442
# time elapsed=8.94[min]|epoch=1|batch=[71/3220]|c-loss=7.56339807510376|s-loss=18.119009017944336|tv-loss=0.0|total loss=25.682407093048095
# time elapsed=10.09[min]|epoch=1|batch=[81/3220]|c-loss=7.397042274475098|s-loss=16.71259536743164|tv-loss=0.0|total loss=24.109637641906737
# time elapsed=11.32[min]|epoch=1|batch=[91/3220]|c-loss=7.445155286788941|s-loss=15.31902904510498|tv-loss=0.0|total loss=22.764184331893922
# time elapsed=12.45[min]|epoch=1|batch=[101/3220]|c-loss=7.0224442958831785|s-loss=14.427418231964111|tv-loss=0.0|total loss=21.44986252784729
# time elapsed=13.58[min]|epoch=1|batch=[111/3220]|c-loss=7.495012187957764|s-loss=13.489839267730712|tv-loss=0.0|total loss=20.984851455688478
# time elapsed=14.73[min]|epoch=1|batch=[121/3220]|c-loss=8.248599243164062|s-loss=12.184775352478027|tv-loss=0.0|total loss=20.43337459564209
# time elapsed=15.84[min]|epoch=1|batch=[131/3220]|c-loss=8.03615345954895|s-loss=11.621710872650146|tv-loss=0.0|total loss=19.657864332199097
# time elapsed=16.96[min]|epoch=1|batch=[141/3220]|c-loss=7.361953544616699|s-loss=10.77902193069458|tv-loss=0.0|total loss=18.14097547531128
# time elapsed=18.08[min]|epoch=1|batch=[151/3220]|c-loss=7.397509479522705|s-loss=10.397507286071777|tv-loss=0.0|total loss=17.795016765594482
# time elapsed=19.18[min]|epoch=1|batch=[161/3220]|c-loss=7.804228210449219|s-loss=10.122551727294923|tv-loss=0.0|total loss=17.92677993774414
# time elapsed=20.31[min]|epoch=1|batch=[171/3220]|c-loss=7.204829359054566|s-loss=9.687411689758301|tv-loss=0.0|total loss=16.892241048812867
# time elapsed=21.41[min]|epoch=1|batch=[181/3220]|c-loss=7.786962366104126|s-loss=9.547326278686523|tv-loss=0.0|total loss=17.33428864479065
# time elapsed=22.53[min]|epoch=1|batch=[191/3220]|c-loss=7.210638093948364|s-loss=9.114473056793212|tv-loss=0.0|total loss=16.325111150741577
# time elapsed=23.72[min]|epoch=1|batch=[201/3220]|c-loss=7.782257461547852|s-loss=8.833004474639893|tv-loss=0.0|total loss=16.615261936187743
# time elapsed=24.86[min]|epoch=1|batch=[211/3220]|c-loss=7.43554196357727|s-loss=8.322647047042846|tv-loss=0.0|total loss=15.758189010620118
# time elapsed=25.99[min]|epoch=1|batch=[221/3220]|c-loss=7.058849859237671|s-loss=7.772788667678833|tv-loss=0.0|total loss=14.831638526916503
# time elapsed=27.06[min]|epoch=1|batch=[231/3220]|c-loss=7.2770164012908936|s-loss=7.529413938522339|tv-loss=0.0|total loss=14.806430339813232
# time elapsed=28.21[min]|epoch=1|batch=[241/3220]|c-loss=7.317415380477906|s-loss=7.236347770690918|tv-loss=0.0|total loss=14.553763151168823
# time elapsed=29.31[min]|epoch=1|batch=[251/3220]|c-loss=7.06897931098938|s-loss=7.076072263717651|tv-loss=0.0|total loss=14.145051574707031
# time elapsed=30.40[min]|epoch=1|batch=[261/3220]|c-loss=7.409995031356812|s-loss=6.733914375305176|tv-loss=0.0|total loss=14.143909406661987
# time elapsed=31.53[min]|epoch=1|batch=[271/3220]|c-loss=6.790026760101318|s-loss=6.385464763641357|tv-loss=0.0|total loss=13.175491523742675
# time elapsed=32.63[min]|epoch=1|batch=[281/3220]|c-loss=7.555556869506836|s-loss=6.457264614105225|tv-loss=0.0|total loss=14.012821483612061
# time elapsed=33.72[min]|epoch=1|batch=[291/3220]|c-loss=7.588534355163574|s-loss=6.171226072311401|tv-loss=0.0|total loss=13.759760427474976
# time elapsed=34.85[min]|epoch=1|batch=[301/3220]|c-loss=6.394856977462768|s-loss=6.449266958236694|tv-loss=0.0|total loss=12.844123935699463
# time elapsed=35.94[min]|epoch=1|batch=[311/3220]|c-loss=7.306825780868531|s-loss=6.285433769226074|tv-loss=0.0|total loss=13.592259550094605
# time elapsed=37.05[min]|epoch=1|batch=[321/3220]|c-loss=7.332931327819824|s-loss=5.961399173736572|tv-loss=0.0|total loss=13.294330501556397
# time elapsed=38.18[min]|epoch=1|batch=[331/3220]|c-loss=7.544522094726562|s-loss=5.8330615043640135|tv-loss=0.0|total loss=13.377583599090576
# time elapsed=39.27[min]|epoch=1|batch=[341/3220]|c-loss=6.754951429367066|s-loss=5.300514841079712|tv-loss=0.0|total loss=12.055466270446777
# time elapsed=40.37[min]|epoch=1|batch=[351/3220]|c-loss=7.189144659042358|s-loss=5.764669513702392|tv-loss=0.0|total loss=12.953814172744751
# time elapsed=41.48[min]|epoch=1|batch=[361/3220]|c-loss=6.745761251449585|s-loss=5.624682760238647|tv-loss=0.0|total loss=12.370444011688232
# time elapsed=42.60[min]|epoch=1|batch=[371/3220]|c-loss=7.3077850341796875|s-loss=5.189078330993652|tv-loss=0.0|total loss=12.49686336517334
# time elapsed=43.70[min]|epoch=1|batch=[381/3220]|c-loss=6.777077388763428|s-loss=5.025319147109985|tv-loss=0.0|total loss=11.802396535873413
# time elapsed=44.83[min]|epoch=1|batch=[391/3220]|c-loss=6.883260202407837|s-loss=4.978359699249268|tv-loss=0.0|total loss=11.861619901657104
# time elapsed=45.94[min]|epoch=1|batch=[401/3220]|c-loss=6.906613159179687|s-loss=4.746778583526611|tv-loss=0.0|total loss=11.6533917427063
# time elapsed=47.06[min]|epoch=1|batch=[411/3220]|c-loss=6.331359481811523|s-loss=4.701555871963501|tv-loss=0.0|total loss=11.032915353775024
# time elapsed=48.19[min]|epoch=1|batch=[421/3220]|c-loss=6.22692494392395|s-loss=4.477744293212891|tv-loss=0.0|total loss=10.70466923713684
# time elapsed=49.33[min]|epoch=1|batch=[431/3220]|c-loss=6.652284097671509|s-loss=4.369922685623169|tv-loss=0.0|total loss=11.022206783294678
# time elapsed=50.51[min]|epoch=1|batch=[441/3220]|c-loss=6.359061574935913|s-loss=4.490850830078125|tv-loss=0.0|total loss=10.849912405014038
# time elapsed=51.61[min]|epoch=1|batch=[451/3220]|c-loss=6.016156339645386|s-loss=4.6017420291900635|tv-loss=0.0|total loss=10.61789836883545
# time elapsed=52.73[min]|epoch=1|batch=[461/3220]|c-loss=5.834393501281738|s-loss=4.664197826385498|tv-loss=0.0|total loss=10.498591327667237





# time elapsed=0.11[min]|epoch=1|batch=[1/1984]|c-loss=0.3735185146331787|cf-loss=0.10768687725067139|s-loss=0.10422005653381347|tv-loss=0.0|total loss=0.5854254484176635
# time elapsed=1.15[min]|epoch=1|batch=[11/1984]|c-loss=3.444767689704895|cf-loss=0.5140327215194702|s-loss=3.6250369787216186|tv-loss=0.0|total loss=7.583837389945984
# time elapsed=0.99[min]|epoch=1|batch=[21/1984]|c-loss=3.6172127723693848|cf-loss=0.5406228512525558|s-loss=1.933218252658844|tv-loss=0.0|total loss=6.091053876280784
# time elapsed=1.13[min]|epoch=1|batch=[31/1984]|c-loss=3.7076677560806273|cf-loss=0.546082180738449|s-loss=1.6308005809783936|tv-loss=0.0|total loss=5.88455051779747
# time elapsed=1.07[min]|epoch=1|batch=[41/1984]|c-loss=3.483558773994446|cf-loss=0.5326883673667908|s-loss=1.5096755385398866|tv-loss=0.0|total loss=5.525922679901123
# time elapsed=1.05[min]|epoch=1|batch=[51/1984]|c-loss=3.5550086975097654|cf-loss=0.5352023363113403|s-loss=1.4738739490509034|tv-loss=0.0|total loss=5.564084982872009
# time elapsed=1.01[min]|epoch=1|batch=[61/1984]|c-loss=3.4939750909805296|cf-loss=0.5319610178470612|s-loss=1.327796220779419|tv-loss=0.0|total loss=5.35373232960701
# time elapsed=1.07[min]|epoch=1|batch=[71/1984]|c-loss=3.4515377998352053|cf-loss=0.5146277546882629|s-loss=1.427315318584442|tv-loss=0.0|total loss=5.39348087310791
# time elapsed=1.01[min]|epoch=1|batch=[81/1984]|c-loss=3.4894356727600098|cf-loss=0.5173875123262406|s-loss=1.4070555210113525|tv-loss=0.0|total loss=5.413878706097603
# time elapsed=1.01[min]|epoch=1|batch=[91/1984]|c-loss=3.4891680479049683|cf-loss=0.5211823225021363|s-loss=1.3812875866889953|tv-loss=0.0|total loss=5.3916379570960995
# time elapsed=1.13[min]|epoch=1|batch=[101/1984]|c-loss=3.4660768270492555|cf-loss=0.5292724072933197|s-loss=1.3957551717758179|tv-loss=0.0|total loss=5.391104406118393
# time elapsed=1.04[min]|epoch=1|batch=[111/1984]|c-loss=3.4466399192810058|cf-loss=0.5120405495166779|s-loss=1.389692759513855|tv-loss=0.0|total loss=5.348373228311539
# time elapsed=1.04[min]|epoch=1|batch=[121/1984]|c-loss=3.430735373497009|cf-loss=0.5047103971242904|s-loss=1.3951369285583497|tv-loss=0.0|total loss=5.330582699179649
# time elapsed=1.09[min]|epoch=1|batch=[131/1984]|c-loss=3.40427451133728|cf-loss=0.5118026018142701|s-loss=1.3964966177940368|tv-loss=0.0|total loss=5.312573730945587
# time elapsed=1.04[min]|epoch=1|batch=[141/1984]|c-loss=3.5263590812683105|cf-loss=0.5029299825429916|s-loss=1.4229375839233398|tv-loss=0.0|total loss=5.452226647734642
# time elapsed=1.04[min]|epoch=1|batch=[151/1984]|c-loss=3.410433602333069|cf-loss=0.5171711921691895|s-loss=1.3435191750526427|tv-loss=0.0|total loss=5.2711239695549015
# time elapsed=1.08[min]|epoch=1|batch=[161/1984]|c-loss=3.4222077131271362|cf-loss=0.5046169996261597|s-loss=1.3457672595977783|tv-loss=0.0|total loss=5.272591972351075
# time elapsed=1.09[min]|epoch=1|batch=[171/1984]|c-loss=3.5198672294616697|cf-loss=0.5095995903015137|s-loss=1.4153235673904419|tv-loss=0.0|total loss=5.444790387153626
# time elapsed=1.11[min]|epoch=1|batch=[181/1984]|c-loss=3.4726642370224|cf-loss=0.5074500620365143|s-loss=1.404535746574402|tv-loss=0.0|total loss=5.384650045633316
# time elapsed=1.11[min]|epoch=1|batch=[191/1984]|c-loss=3.510091495513916|cf-loss=0.4962306171655655|s-loss=1.4068424344062804|tv-loss=0.0|total loss=5.413164547085762
# time elapsed=0.98[min]|epoch=1|batch=[201/1984]|c-loss=3.431357717514038|cf-loss=0.500407999753952|s-loss=1.3918726444244385|tv-loss=0.0|total loss=5.323638361692429
# time elapsed=1.14[min]|epoch=1|batch=[211/1984]|c-loss=3.4570037841796877|cf-loss=0.5083957284688949|s-loss=1.3529536485671998|tv-loss=0.0|total loss=5.318353161215782
# time elapsed=1.11[min]|epoch=1|batch=[221/1984]|c-loss=3.4193569660186767|cf-loss=0.5004053562879562|s-loss=1.3790010809898376|tv-loss=0.0|total loss=5.298763403296471
# time elapsed=1.09[min]|epoch=1|batch=[231/1984]|c-loss=3.482963275909424|cf-loss=0.5030550807714462|s-loss=1.3847736120224|tv-loss=0.0|total loss=5.37079196870327
# time elapsed=1.14[min]|epoch=1|batch=[241/1984]|c-loss=3.4178353786468505|cf-loss=0.4971780747175217|s-loss=1.385081684589386|tv-loss=0.0|total loss=5.300095137953758


In [ ]:
# TODO
# DONE # 1. Select images from Widerface with (1-4)small number of faces
# DONE # 2. Load existing saved weights and then train on WIDERFACE
# 3. Additional Loss for Content of Face at the output of transformer net
# 4. Prevent styling of face -> Loss minimization helps in styling -> Prevent Styling on Face locations -> Subtract "masked" loss in gram matrix

In [ ]:
# Dataset Size :  7935
# Using 15872 datapoints (3968 batches) (MS COCO images) for transformer network training.
# Loaded !
# Learning the style of edtaonisl.jpg style image.
# ********************************************************************************
# Hyperparams: content_weight=1.0, style_weight=400000.0 and tv_weight=0
# ********************************************************************************
# Logging to console every 10 batches.
# Saving checkpoint models every 100 batches.
# Tensorboard disabled.
# ********************************************************************************
# time elapsed=0.13[min]|epoch=1|batch=[1/1984]|c-loss=0.4011974334716797|cf-loss=0.006361397355794907|s-loss=0.10607430934906006|tv-loss=0.0|total loss=0.5136331401765346
# time elapsed=1.16[min]|epoch=1|batch=[11/1984]|c-loss=3.1180899381637572|cf-loss=0.0599148671142757|s-loss=15.14263882637024|tv-loss=0.0|total loss=18.320643631648274
# time elapsed=1.20[min]|epoch=1|batch=[21/1984]|c-loss=3.3385652780532835|cf-loss=0.04407721441239119|s-loss=5.626981401443482|tv-loss=0.0|total loss=9.009623893909156
# time elapsed=1.14[min]|epoch=1|batch=[31/1984]|c-loss=3.586587405204773|cf-loss=0.020866224868223072|s-loss=3.494497203826904|tv-loss=0.0|total loss=7.101950833899901
# time elapsed=1.15[min]|epoch=1|batch=[41/1984]|c-loss=3.645912456512451|cf-loss=0.0316261924803257|s-loss=2.4224989891052244|tv-loss=0.0|total loss=6.1000376380980015
# time elapsed=1.21[min]|epoch=1|batch=[51/1984]|c-loss=3.6648753881454468|cf-loss=0.040337744355201724|s-loss=1.9301427245140075|tv-loss=0.0|total loss=5.635355857014656
# time elapsed=1.21[min]|epoch=1|batch=[61/1984]|c-loss=3.6702167272567747|cf-loss=0.060292269289493564|s-loss=1.7013561964035033|tv-loss=0.0|total loss=5.431865192949772
# time elapsed=1.19[min]|epoch=1|batch=[71/1984]|c-loss=3.754121112823486|cf-loss=0.04029329461045563|s-loss=1.540630292892456|tv-loss=0.0|total loss=5.335044700326398
# time elapsed=1.17[min]|epoch=1|batch=[81/1984]|c-loss=3.6607953310012817|cf-loss=0.04301696224138141|s-loss=1.474129033088684|tv-loss=0.0|total loss=5.177941326331347
# time elapsed=1.20[min]|epoch=1|batch=[91/1984]|c-loss=3.692770528793335|cf-loss=0.04627019250765443|s-loss=1.4733418464660644|tv-loss=0.0|total loss=5.212382567767054
# time elapsed=1.18[min]|epoch=1|batch=[101/1984]|c-loss=3.6605846881866455|cf-loss=0.06852411478757858|s-loss=1.402559518814087|tv-loss=0.0|total loss=5.131668321788311
# time elapsed=1.32[min]|epoch=1|batch=[111/1984]|c-loss=3.5688848733901977|cf-loss=0.02722410480491817|s-loss=1.3526429653167724|tv-loss=0.0|total loss=4.948751943511889
# time elapsed=1.27[min]|epoch=1|batch=[121/1984]|c-loss=3.623447632789612|cf-loss=0.05797116626054048|s-loss=1.3597532391548157|tv-loss=0.0|total loss=5.041172038204968
# time elapsed=1.32[min]|epoch=1|batch=[131/1984]|c-loss=3.5943480253219606|cf-loss=0.06452927570790053|s-loss=1.285047745704651|tv-loss=0.0|total loss=4.9439250467345115
# time elapsed=1.21[min]|epoch=1|batch=[141/1984]|c-loss=3.597687268257141|cf-loss=0.06521459762006998|s-loss=1.2846673250198364|tv-loss=0.0|total loss=4.947569190897047
# time elapsed=1.13[min]|epoch=1|batch=[151/1984]|c-loss=3.6015135049819946|cf-loss=0.047717547649517654|s-loss=1.327004647254944|tv-loss=0.0|total loss=4.976235699886456
# time elapsed=1.25[min]|epoch=1|batch=[161/1984]|c-loss=3.5814526081085205|cf-loss=0.040486734732985495|s-loss=1.2449369788169862|tv-loss=0.0|total loss=4.866876321658492
# time elapsed=1.17[min]|epoch=1|batch=[171/1984]|c-loss=3.549436616897583|cf-loss=0.05281154215335846|s-loss=1.2310694694519042|tv-loss=0.0|total loss=4.833317628502845
# time elapsed=1.12[min]|epoch=1|batch=[181/1984]|c-loss=3.5483327388763426|cf-loss=0.02758824694901705|s-loss=1.2332257270812987|tv-loss=0.0|total loss=4.809146712906658
# time elapsed=1.26[min]|epoch=1|batch=[191/1984]|c-loss=3.5611549377441407|cf-loss=0.044269778905436395|s-loss=1.2331366777420043|tv-loss=0.0|total loss=4.838561394391581
# time elapsed=1.17[min]|epoch=1|batch=[201/1984]|c-loss=3.5517608642578127|cf-loss=0.04539442993700504|s-loss=1.1952430367469788|tv-loss=0.0|total loss=4.792398330941796
# time elapsed=1.19[min]|epoch=1|batch=[211/1984]|c-loss=3.537069296836853|cf-loss=0.08238174207508564|s-loss=1.2336463928222656|tv-loss=0.0|total loss=4.853097431734204
# time elapsed=1.14[min]|epoch=1|batch=[221/1984]|c-loss=3.4847748041152955|cf-loss=0.0451109716668725|s-loss=1.2266721248626709|tv-loss=0.0|total loss=4.756557900644839
# time elapsed=1.12[min]|epoch=1|batch=[231/1984]|c-loss=3.553849220275879|cf-loss=0.06058542467653751|s-loss=1.2270436763763428|tv-loss=0.0|total loss=4.8414783213287595
# time elapsed=1.18[min]|epoch=1|batch=[241/1984]|c-loss=3.5229861974716186|cf-loss=0.04575381623581052|s-loss=1.2050271689891816|tv-loss=0.0|total loss=4.773767182696611
# time elapsed=1.15[min]|epoch=1|batch=[251/1984]|c-loss=3.5441877603530885|cf-loss=0.05925847264006734|s-loss=1.1930065870285034|tv-loss=0.0|total loss=4.796452820021659
# time elapsed=1.19[min]|epoch=1|batch=[261/1984]|c-loss=3.5540643453598024|cf-loss=0.04481137003749609|s-loss=1.2074613332748414|tv-loss=0.0|total loss=4.806337048672139
# time elapsed=1.13[min]|epoch=1|batch=[271/1984]|c-loss=3.4933244943618775|cf-loss=0.03845018055289984|s-loss=1.1987479686737061|tv-loss=0.0|total loss=4.7305226435884835
# time elapsed=1.18[min]|epoch=1|batch=[281/1984]|c-loss=3.5532256603240966|cf-loss=0.05348817361518741|s-loss=1.2648723721504211|tv-loss=0.0|total loss=4.871586206089705
# time elapsed=1.18[min]|epoch=1|batch=[291/1984]|c-loss=3.4790741443634032|cf-loss=0.05367026086896658|s-loss=1.1565542340278625|tv-loss=0.0|total loss=4.689298639260232
# time elapsed=1.21[min]|epoch=1|batch=[301/1984]|c-loss=3.5212577104568483|cf-loss=0.03894479032605887|s-loss=1.1894597172737122|tv-loss=0.0|total loss=4.749662218056619
# time elapsed=1.21[min]|epoch=1|batch=[311/1984]|c-loss=3.5199583768844604|cf-loss=0.04640936963260174|s-loss=1.1938647985458375|tv-loss=0.0|total loss=4.7602325450629
# time elapsed=1.20[min]|epoch=1|batch=[321/1984]|c-loss=3.501480150222778|cf-loss=0.05139649948105216|s-loss=1.2049229502677918|tv-loss=0.0|total loss=4.757799599971622
# time elapsed=1.25[min]|epoch=1|batch=[331/1984]|c-loss=3.496416473388672|cf-loss=0.05137581378221512|s-loss=1.1593147456645965|tv-loss=0.0|total loss=4.707107032835483
# time elapsed=1.17[min]|epoch=1|batch=[341/1984]|c-loss=3.5251781702041627|cf-loss=0.06399255432188511|s-loss=1.1449594020843505|tv-loss=0.0|total loss=4.734130126610398
# time elapsed=1.20[min]|epoch=1|batch=[351/1984]|c-loss=3.500110960006714|cf-loss=0.054848956502974035|s-loss=1.1567005157470702|tv-loss=0.0|total loss=4.711660432256759
# time elapsed=1.18[min]|epoch=1|batch=[361/1984]|c-loss=3.4943788766860964|cf-loss=0.05289692040532827|s-loss=1.1828686594963074|tv-loss=0.0|total loss=4.730144456587732
# time elapsed=1.21[min]|epoch=1|batch=[371/1984]|c-loss=3.468537712097168|cf-loss=0.05114356894046068|s-loss=1.1457360148429871|tv-loss=0.0|total loss=4.665417295880616
# time elapsed=1.19[min]|epoch=1|batch=[381/1984]|c-loss=3.5620824813842775|cf-loss=0.06850692238658666|s-loss=1.2337618470191956|tv-loss=0.0|total loss=4.86435125079006
# time elapsed=1.15[min]|epoch=1|batch=[391/1984]|c-loss=3.4404978275299074|cf-loss=0.040320449881255625|s-loss=1.178521466255188|tv-loss=0.0|total loss=4.659339743666351
# time elapsed=1.19[min]|epoch=1|batch=[401/1984]|c-loss=3.516886901855469|cf-loss=0.03058893084526062|s-loss=1.1667979836463929|tv-loss=0.0|total loss=4.714273816347122
# time elapsed=1.18[min]|epoch=1|batch=[411/1984]|c-loss=3.4231775760650636|cf-loss=0.03441429315134883|s-loss=1.1458013892173766|tv-loss=0.0|total loss=4.603393258433789
# time elapsed=1.25[min]|epoch=1|batch=[421/1984]|c-loss=3.48213050365448|cf-loss=0.05428443532437086|s-loss=1.2095076441764832|tv-loss=0.0|total loss=4.745922583155334
# time elapsed=1.21[min]|epoch=1|batch=[431/1984]|c-loss=3.522844171524048|cf-loss=0.03500297348946333|s-loss=1.193105947971344|tv-loss=0.0|total loss=4.750953092984855
# time elapsed=1.22[min]|epoch=1|batch=[441/1984]|c-loss=3.556228232383728|cf-loss=0.06697033350355923|s-loss=1.2344560027122498|tv-loss=0.0|total loss=4.857654568599537
# time elapsed=1.16[min]|epoch=1|batch=[451/1984]|c-loss=3.5951178789138796|cf-loss=0.07024158518761396|s-loss=1.2007521152496339|tv-loss=0.0|total loss=4.866111579351127
# time elapsed=1.15[min]|epoch=1|batch=[461/1984]|c-loss=3.5482211351394652|cf-loss=0.07566336295567452|s-loss=1.1504177689552306|tv-loss=0.0|total loss=4.774302267050371
# time elapsed=1.21[min]|epoch=1|batch=[471/1984]|c-loss=3.4843114614486694|cf-loss=0.04015296716243029|s-loss=1.1352127194404602|tv-loss=0.0|total loss=4.65967714805156
# time elapsed=1.22[min]|epoch=1|batch=[481/1984]|c-loss=3.524331784248352|cf-loss=0.06541303861886263|s-loss=1.150844931602478|tv-loss=0.0|total loss=4.740589754469693
# time elapsed=1.22[min]|epoch=1|batch=[491/1984]|c-loss=3.4699280977249147|cf-loss=0.06839211396872998|s-loss=1.131784051656723|tv-loss=0.0|total loss=4.6701042633503675
# time elapsed=1.13[min]|epoch=1|batch=[501/1984]|c-loss=3.470432162284851|cf-loss=0.059911233745515345|s-loss=1.1797105550765992|tv-loss=0.0|total loss=4.710053951106966
# time elapsed=1.15[min]|epoch=1|batch=[511/1984]|c-loss=3.5096673011779784|cf-loss=0.05487087378278375|s-loss=1.1739107489585876|tv-loss=0.0|total loss=4.73844892391935
# time elapsed=1.24[min]|epoch=1|batch=[521/1984]|c-loss=3.5379156351089476|cf-loss=0.06896273251622916|s-loss=1.1515349268913269|tv-loss=0.0|total loss=4.758413294516504
# time elapsed=1.14[min]|epoch=1|batch=[531/1984]|c-loss=3.498807501792908|cf-loss=0.051736618485301734|s-loss=1.119760310649872|tv-loss=0.0|total loss=4.6703044309280815
# time elapsed=1.18[min]|epoch=1|batch=[541/1984]|c-loss=3.4343817710876463|cf-loss=0.07007119674235582|s-loss=1.1414728045463562|tv-loss=0.0|total loss=4.6459257723763585
# time elapsed=1.21[min]|epoch=1|batch=[551/1984]|c-loss=3.5213319063186646|cf-loss=0.04052512990310788|s-loss=1.1793827414512634|tv-loss=0.0|total loss=4.7412397776730355
# time elapsed=1.16[min]|epoch=1|batch=[561/1984]|c-loss=3.444315767288208|cf-loss=0.0476468020118773|s-loss=1.1207151651382445|tv-loss=0.0|total loss=4.61267773443833
# time elapsed=1.20[min]|epoch=1|batch=[571/1984]|c-loss=3.4492029666900637|cf-loss=0.03691104063764215|s-loss=1.1546232938766479|tv-loss=0.0|total loss=4.640737301204354
# time elapsed=1.19[min]|epoch=1|batch=[581/1984]|c-loss=3.472251224517822|cf-loss=0.052284382097423074|s-loss=1.1339638352394104|tv-loss=0.0|total loss=4.658499441854656
# time elapsed=1.13[min]|epoch=1|batch=[591/1984]|c-loss=3.386936402320862|cf-loss=0.023052756022661925|s-loss=1.1086387157440185|tv-loss=0.0|total loss=4.518627874087542
# time elapsed=1.16[min]|epoch=1|batch=[601/1984]|c-loss=3.4396220684051513|cf-loss=0.02759717726148665|s-loss=1.1235631465911866|tv-loss=0.0|total loss=4.590782392257824
# time elapsed=1.16[min]|epoch=1|batch=[611/1984]|c-loss=3.455087089538574|cf-loss=0.06927001336589456|s-loss=1.1678065657615662|tv-loss=0.0|total loss=4.692163668666035
# time elapsed=1.20[min]|epoch=1|batch=[621/1984]|c-loss=3.4876052379608153|cf-loss=0.060692410729825494|s-loss=1.1811116218566895|tv-loss=0.0|total loss=4.729409270547331
# time elapsed=1.20[min]|epoch=1|batch=[631/1984]|c-loss=3.437530541419983|cf-loss=0.05992977097630501|s-loss=1.1669270515441894|tv-loss=0.0|total loss=4.664387363940477
# time elapsed=1.24[min]|epoch=1|batch=[641/1984]|c-loss=3.42463276386261|cf-loss=0.0434399489313364|s-loss=1.1366382241249084|tv-loss=0.0|total loss=4.604710936918854
# time elapsed=1.15[min]|epoch=1|batch=[651/1984]|c-loss=3.484876585006714|cf-loss=0.05901594143360853|s-loss=1.1675257086753845|tv-loss=0.0|total loss=4.711418235115707
# time elapsed=1.22[min]|epoch=1|batch=[661/1984]|c-loss=3.431078863143921|cf-loss=0.04354045987129211|s-loss=1.1725112915039062|tv-loss=0.0|total loss=4.647130614519119
# time elapsed=1.19[min]|epoch=1|batch=[671/1984]|c-loss=3.495618462562561|cf-loss=0.05971381282433867|s-loss=1.1804605841636657|tv-loss=0.0|total loss=4.735792859550566
# time elapsed=1.17[min]|epoch=1|batch=[681/1984]|c-loss=3.4624711990356447|cf-loss=0.04884469453245401|s-loss=1.1422843217849732|tv-loss=0.0|total loss=4.653600215353071
# time elapsed=1.26[min]|epoch=1|batch=[691/1984]|c-loss=3.4772079229354858|cf-loss=0.04855214916169644|s-loss=1.1470025539398194|tv-loss=0.0|total loss=4.672762626037001
# time elapsed=1.20[min]|epoch=1|batch=[701/1984]|c-loss=3.452958846092224|cf-loss=0.0648894427344203|s-loss=1.1401103615760804|tv-loss=0.0|total loss=4.657958650402724
# time elapsed=1.21[min]|epoch=1|batch=[711/1984]|c-loss=3.46107873916626|cf-loss=0.04843868813477457|s-loss=1.1251832246780396|tv-loss=0.0|total loss=4.634700651979074
# time elapsed=1.16[min]|epoch=1|batch=[721/1984]|c-loss=3.3716142654418944|cf-loss=0.030125437397509815|s-loss=1.1526132583618165|tv-loss=0.0|total loss=4.554352961201221
# time elapsed=1.22[min]|epoch=1|batch=[731/1984]|c-loss=3.4734599351882935|cf-loss=0.04966266583651304|s-loss=1.114137303829193|tv-loss=0.0|total loss=4.637259904854
# time elapsed=1.19[min]|epoch=1|batch=[741/1984]|c-loss=3.3906688690185547|cf-loss=0.05850357562303543|s-loss=1.140290880203247|tv-loss=0.0|total loss=4.5894633248448375
# time elapsed=1.23[min]|epoch=1|batch=[751/1984]|c-loss=3.4799007415771483|cf-loss=0.05763514433056116|s-loss=1.1505884170532226|tv-loss=0.0|total loss=4.688124302960932
# time elapsed=1.22[min]|epoch=1|batch=[761/1984]|c-loss=3.3791232347488402|cf-loss=0.0354842814616859|s-loss=1.1885722160339356|tv-loss=0.0|total loss=4.603179732244461
# time elapsed=1.10[min]|epoch=1|batch=[771/1984]|c-loss=3.443797469139099|cf-loss=0.04814870823174715|s-loss=1.1221470355987548|tv-loss=0.0|total loss=4.6140932129696015
# time elapsed=1.23[min]|epoch=1|batch=[781/1984]|c-loss=3.5055933952331544|cf-loss=0.05155434384942055|s-loss=1.1509586572647095|tv-loss=0.0|total loss=4.708106396347285
# time elapsed=1.16[min]|epoch=1|batch=[791/1984]|c-loss=3.5212491035461424|cf-loss=0.032420352147892115|s-loss=1.1655795216560363|tv-loss=0.0|total loss=4.719248977350071
# time elapsed=1.15[min]|epoch=1|batch=[801/1984]|c-loss=3.488077425956726|cf-loss=0.032299921941012145|s-loss=1.195937180519104|tv-loss=0.0|total loss=4.716314528416842
# time elapsed=1.21[min]|epoch=1|batch=[811/1984]|c-loss=3.4162309646606444|cf-loss=0.030169572029262782|s-loss=1.1659811973571776|tv-loss=0.0|total loss=4.612381734047085
# time elapsed=1.12[min]|epoch=1|batch=[821/1984]|c-loss=3.38195641040802|cf-loss=0.04089138209819794|s-loss=1.125494647026062|tv-loss=0.0|total loss=4.54834243953228
# time elapsed=1.13[min]|epoch=1|batch=[831/1984]|c-loss=3.417183828353882|cf-loss=0.047304294724017384|s-loss=1.1662765622138977|tv-loss=0.0|total loss=4.630764685291797
# time elapsed=1.21[min]|epoch=1|batch=[841/1984]|c-loss=3.453539228439331|cf-loss=0.047925705648958684|s-loss=1.1371261358261109|tv-loss=0.0|total loss=4.6385910699144
# time elapsed=1.18[min]|epoch=1|batch=[851/1984]|c-loss=3.396494245529175|cf-loss=0.07677428945899009|s-loss=1.1325582981109619|tv-loss=0.0|total loss=4.605826833099127
# time elapsed=1.16[min]|epoch=1|batch=[861/1984]|c-loss=3.5149884939193727|cf-loss=0.044644671771675346|s-loss=1.199470865726471|tv-loss=0.0|total loss=4.759104031417519
# time elapsed=1.19[min]|epoch=1|batch=[871/1984]|c-loss=3.4635622262954713|cf-loss=0.04799373745918274|s-loss=1.1908321380615234|tv-loss=0.0|total loss=4.702388101816178
# time elapsed=1.21[min]|epoch=1|batch=[881/1984]|c-loss=3.464625024795532|cf-loss=0.04224623483605683|s-loss=1.1336945414543151|tv-loss=0.0|total loss=4.640565801085904
# time elapsed=1.13[min]|epoch=1|batch=[891/1984]|c-loss=3.4988354444503784|cf-loss=0.03164411140605807|s-loss=1.1557652473449707|tv-loss=0.0|total loss=4.686244803201407
# time elapsed=1.15[min]|epoch=1|batch=[901/1984]|c-loss=3.39470579624176|cf-loss=0.06648878594860434|s-loss=1.1355177998542785|tv-loss=0.0|total loss=4.596712382044643
# time elapsed=1.19[min]|epoch=1|batch=[911/1984]|c-loss=3.4339976787567137|cf-loss=0.030955934338271617|s-loss=1.1714810132980347|tv-loss=0.0|total loss=4.6364346263930205
# time elapsed=1.12[min]|epoch=1|batch=[921/1984]|c-loss=3.3834697008132935|cf-loss=0.03882670626044273|s-loss=1.1410748481750488|tv-loss=0.0|total loss=4.563371255248785
# time elapsed=1.18[min]|epoch=1|batch=[931/1984]|c-loss=3.5137878894805907|cf-loss=0.03165988507680595|s-loss=1.1210232734680177|tv-loss=0.0|total loss=4.666471048025414
# time elapsed=1.16[min]|epoch=1|batch=[941/1984]|c-loss=3.5054585456848146|cf-loss=0.047740844264626506|s-loss=1.1545461654663085|tv-loss=0.0|total loss=4.70774555541575
# time elapsed=1.20[min]|epoch=1|batch=[951/1984]|c-loss=3.410657787322998|cf-loss=0.05463231652975083|s-loss=1.1604772567749024|tv-loss=0.0|total loss=4.625767360627651
# time elapsed=1.17[min]|epoch=1|batch=[961/1984]|c-loss=3.495836281776428|cf-loss=0.052805589605122806|s-loss=1.1347675919532776|tv-loss=0.0|total loss=4.683409463334828
# time elapsed=1.17[min]|epoch=1|batch=[971/1984]|c-loss=3.5391701221466065|cf-loss=0.05184876509010792|s-loss=1.1677669882774353|tv-loss=0.0|total loss=4.75878587551415
# time elapsed=1.14[min]|epoch=1|batch=[981/1984]|c-loss=3.4679232120513914|cf-loss=0.04063983429223299|s-loss=1.1235924363136292|tv-loss=0.0|total loss=4.632155482657254
# time elapsed=1.16[min]|epoch=1|batch=[991/1984]|c-loss=3.45146164894104|cf-loss=0.056992799043655396|s-loss=1.163216507434845|tv-loss=0.0|total loss=4.671670955419541
# time elapsed=1.23[min]|epoch=1|batch=[1001/1984]|c-loss=3.4514906883239744|cf-loss=0.05551575776189566|s-loss=1.133684015274048|tv-loss=0.0|total loss=4.6406904613599185
# time elapsed=1.22[min]|epoch=1|batch=[1011/1984]|c-loss=3.4676076889038088|cf-loss=0.04457811415195465|s-loss=1.1210680246353149|tv-loss=0.0|total loss=4.633253827691078
# time elapsed=1.23[min]|epoch=1|batch=[1021/1984]|c-loss=3.4701639890670775|cf-loss=0.03850771291181445|s-loss=1.1722778260707856|tv-loss=0.0|total loss=4.680949528049678
# time elapsed=1.18[min]|epoch=1|batch=[1031/1984]|c-loss=3.4425976276397705|cf-loss=0.06641724910587073|s-loss=1.1183662295341492|tv-loss=0.0|total loss=4.62738110627979
# time elapsed=1.16[min]|epoch=1|batch=[1041/1984]|c-loss=3.47123703956604|cf-loss=0.05127592189237475|s-loss=1.1199093639850617|tv-loss=0.0|total loss=4.642422325443476
# time elapsed=1.18[min]|epoch=1|batch=[1051/1984]|c-loss=3.437276101112366|cf-loss=0.05662264227867127|s-loss=1.204825723171234|tv-loss=0.0|total loss=4.698724466562271
# time elapsed=1.21[min]|epoch=1|batch=[1061/1984]|c-loss=3.462959909439087|cf-loss=0.06952195707708597|s-loss=1.1911860346794128|tv-loss=0.0|total loss=4.7236679011955856
# time elapsed=1.21[min]|epoch=1|batch=[1071/1984]|c-loss=3.4049349308013914|cf-loss=0.05129965078085661|s-loss=1.117117702960968|tv-loss=0.0|total loss=4.573352284543216
# time elapsed=1.18[min]|epoch=1|batch=[1081/1984]|c-loss=3.421842432022095|cf-loss=0.06498414995148778|s-loss=1.1444974064826965|tv-loss=0.0|total loss=4.631323988456279
# time elapsed=1.20[min]|epoch=1|batch=[1091/1984]|c-loss=3.409850573539734|cf-loss=0.055317046120762825|s-loss=1.1661778450012208|tv-loss=0.0|total loss=4.631345464661718
# time elapsed=1.16[min]|epoch=1|batch=[1101/1984]|c-loss=3.4876943588256837|cf-loss=0.033825787156820296|s-loss=1.1242737412452697|tv-loss=0.0|total loss=4.645793887227773
# time elapsed=1.17[min]|epoch=1|batch=[1111/1984]|c-loss=3.441155481338501|cf-loss=0.06323359198868275|s-loss=1.1636282205581665|tv-loss=0.0|total loss=4.668017293885351
# time elapsed=1.19[min]|epoch=1|batch=[1121/1984]|c-loss=3.404826354980469|cf-loss=0.05401105862110853|s-loss=1.1348465085029602|tv-loss=0.0|total loss=4.593683922104537
# time elapsed=1.18[min]|epoch=1|batch=[1131/1984]|c-loss=3.4829078674316407|cf-loss=0.03805298507213593|s-loss=1.1447667479515076|tv-loss=0.0|total loss=4.665727600455284
# time elapsed=1.26[min]|epoch=1|batch=[1141/1984]|c-loss=3.5008056640625|cf-loss=0.05338873490691185|s-loss=1.1603332042694092|tv-loss=0.0|total loss=4.714527603238821
# time elapsed=1.14[min]|epoch=1|batch=[1151/1984]|c-loss=3.468635845184326|cf-loss=0.03647188190370798|s-loss=1.1700696349143982|tv-loss=0.0|total loss=4.675177362002432
# time elapsed=1.18[min]|epoch=1|batch=[1161/1984]|c-loss=3.4449716806411743|cf-loss=0.04702179878950119|s-loss=1.192429006099701|tv-loss=0.0|total loss=4.684422485530376
# time elapsed=1.21[min]|epoch=1|batch=[1171/1984]|c-loss=3.4783188581466673|cf-loss=0.09745364505797624|s-loss=1.1364899396896362|tv-loss=0.0|total loss=4.71226244289428
# time elapsed=1.13[min]|epoch=1|batch=[1181/1984]|c-loss=3.4845504999160766|cf-loss=0.08053413527086377|s-loss=1.1520126461982727|tv-loss=0.0|total loss=4.717097281385213
# time elapsed=1.16[min]|epoch=1|batch=[1191/1984]|c-loss=3.375800037384033|cf-loss=0.0578147841617465|s-loss=1.1453263878822326|tv-loss=0.0|total loss=4.578941209428012
# time elapsed=1.23[min]|epoch=1|batch=[1201/1984]|c-loss=3.5071431159973145|cf-loss=0.03495584344491363|s-loss=1.194425928592682|tv-loss=0.0|total loss=4.73652488803491
# time elapsed=1.20[min]|epoch=1|batch=[1211/1984]|c-loss=3.4460309982299804|cf-loss=0.06572731155902148|s-loss=1.186260747909546|tv-loss=0.0|total loss=4.698019057698548
# time elapsed=1.19[min]|epoch=1|batch=[1221/1984]|c-loss=3.5065582275390623|cf-loss=0.030227346159517766|s-loss=1.139247727394104|tv-loss=0.0|total loss=4.6760333010926844
# time elapsed=1.20[min]|epoch=1|batch=[1231/1984]|c-loss=3.4072701215744017|cf-loss=0.050619575474411246|s-loss=1.186532163619995|tv-loss=0.0|total loss=4.644421860668809
# time elapsed=1.23[min]|epoch=1|batch=[1241/1984]|c-loss=3.4752664089202883|cf-loss=0.05548178348690271|s-loss=1.1983989000320434|tv-loss=0.0|total loss=4.729147092439234
# time elapsed=1.10[min]|epoch=1|batch=[1251/1984]|c-loss=3.4039101600646973|cf-loss=0.035502315126359464|s-loss=1.172924852371216|tv-loss=0.0|total loss=4.6123373275622725
# time elapsed=1.17[min]|epoch=1|batch=[1261/1984]|c-loss=3.493249750137329|cf-loss=0.05520501416176558|s-loss=1.13382950425148|tv-loss=0.0|total loss=4.682284268550575
# time elapsed=1.19[min]|epoch=1|batch=[1271/1984]|c-loss=3.3917540073394776|cf-loss=0.028948307828977705|s-loss=1.1851593852043152|tv-loss=0.0|total loss=4.60586170037277
# time elapsed=1.17[min]|epoch=1|batch=[1281/1984]|c-loss=3.3982475996017456|cf-loss=0.05338263940066099|s-loss=1.1317143201828004|tv-loss=0.0|total loss=4.583344559185207
# time elapsed=1.25[min]|epoch=1|batch=[1291/1984]|c-loss=3.5067672729492188|cf-loss=0.04315538024529815|s-loss=1.1566625952720642|tv-loss=0.0|total loss=4.706585248466581
# time elapsed=1.23[min]|epoch=1|batch=[1301/1984]|c-loss=3.5412106037139894|cf-loss=0.04410902224481106|s-loss=1.1729864835739137|tv-loss=0.0|total loss=4.758306109532714
# time elapsed=1.16[min]|epoch=1|batch=[1311/1984]|c-loss=3.4988972187042235|cf-loss=0.05041662324219942|s-loss=1.108347588777542|tv-loss=0.0|total loss=4.657661430723965
# time elapsed=1.19[min]|epoch=1|batch=[1321/1984]|c-loss=3.4151933908462526|cf-loss=0.060417629219591616|s-loss=1.1286859035491943|tv-loss=0.0|total loss=4.604296923615038
# time elapsed=1.16[min]|epoch=1|batch=[1331/1984]|c-loss=3.4870884895324705|cf-loss=0.04869463872164488|s-loss=1.1524391412734984|tv-loss=0.0|total loss=4.688222269527614
# time elapsed=1.17[min]|epoch=1|batch=[1341/1984]|c-loss=3.455456757545471|cf-loss=0.038279902469366786|s-loss=1.1122326612472535|tv-loss=0.0|total loss=4.605969321262092
# time elapsed=1.18[min]|epoch=1|batch=[1351/1984]|c-loss=3.490167021751404|cf-loss=0.04171179668046534|s-loss=1.1429193794727326|tv-loss=0.0|total loss=4.674798197904602
# time elapsed=1.23[min]|epoch=1|batch=[1361/1984]|c-loss=3.3813810110092164|cf-loss=0.03315602610819042|s-loss=1.1097333312034607|tv-loss=0.0|total loss=4.524270368320868
# time elapsed=1.22[min]|epoch=1|batch=[1371/1984]|c-loss=3.422178363800049|cf-loss=0.034107699804008004|s-loss=1.180924367904663|tv-loss=0.0|total loss=4.63721043150872
# time elapsed=1.14[min]|epoch=1|batch=[1381/1984]|c-loss=3.5152349710464477|cf-loss=0.04746650117449462|s-loss=1.1353960394859315|tv-loss=0.0|total loss=4.698097511706874
# time elapsed=1.20[min]|epoch=1|batch=[1391/1984]|c-loss=3.4487663984298704|cf-loss=0.05180081180296838|s-loss=1.141047203540802|tv-loss=0.0|total loss=4.641614413773641
# time elapsed=1.20[min]|epoch=1|batch=[1401/1984]|c-loss=3.4287746429443358|cf-loss=0.049665738455951215|s-loss=1.1400225281715393|tv-loss=0.0|total loss=4.618462909571827
# time elapsed=1.25[min]|epoch=1|batch=[1411/1984]|c-loss=3.4774820804595947|cf-loss=0.06273295748978854|s-loss=1.16270272731781|tv-loss=0.0|total loss=4.702917765267193
# time elapsed=1.12[min]|epoch=1|batch=[1421/1984]|c-loss=3.4919243812561036|cf-loss=0.04027510015293956|s-loss=1.13463693857193|tv-loss=0.0|total loss=4.666836419980973
# time elapsed=1.20[min]|epoch=1|batch=[1431/1984]|c-loss=3.42773973941803|cf-loss=0.04415282839909196|s-loss=1.147992205619812|tv-loss=0.0|total loss=4.619884773436934
# time elapsed=1.24[min]|epoch=1|batch=[1441/1984]|c-loss=3.477733540534973|cf-loss=0.032536485325545074|s-loss=1.1494537115097045|tv-loss=0.0|total loss=4.659723737370223
# time elapsed=1.22[min]|epoch=1|batch=[1451/1984]|c-loss=3.393999147415161|cf-loss=0.03353941170498729|s-loss=1.1165900111198426|tv-loss=0.0|total loss=4.544128570239991
# time elapsed=1.09[min]|epoch=1|batch=[1461/1984]|c-loss=3.4726011991500854|cf-loss=0.030909512750804426|s-loss=1.1506307601928711|tv-loss=0.0|total loss=4.654141472093761
# time elapsed=1.16[min]|epoch=1|batch=[1471/1984]|c-loss=3.447489595413208|cf-loss=0.05380089171230793|s-loss=1.1757465660572053|tv-loss=0.0|total loss=4.677037053182721
# time elapsed=1.21[min]|epoch=1|batch=[1481/1984]|c-loss=3.489210081100464|cf-loss=0.05271808179095387|s-loss=1.1473976612091064|tv-loss=0.0|total loss=4.689325824100524
# time elapsed=1.21[min]|epoch=1|batch=[1491/1984]|c-loss=3.425186562538147|cf-loss=0.03748173294588923|s-loss=1.1258513808250428|tv-loss=0.0|total loss=4.588519676309079
# time elapsed=1.22[min]|epoch=1|batch=[1501/1984]|c-loss=3.4659393548965456|cf-loss=0.04294793615117669|s-loss=1.1361870288848877|tv-loss=0.0|total loss=4.645074319932609
# time elapsed=1.20[min]|epoch=1|batch=[1511/1984]|c-loss=3.4238343715667723|cf-loss=0.038209553342312574|s-loss=1.1442631006240844|tv-loss=0.0|total loss=4.6063070255331695
# time elapsed=1.13[min]|epoch=1|batch=[1521/1984]|c-loss=3.4259344339370728|cf-loss=0.04766311701387167|s-loss=1.1258261561393739|tv-loss=0.0|total loss=4.599423707090319
# time elapsed=1.13[min]|epoch=1|batch=[1531/1984]|c-loss=3.421577310562134|cf-loss=0.03581875483505428|s-loss=1.158464789390564|tv-loss=0.0|total loss=4.615860854787752
# time elapsed=1.23[min]|epoch=1|batch=[1541/1984]|c-loss=3.462989640235901|cf-loss=0.05774399153888225|s-loss=1.1213556945323944|tv-loss=0.0|total loss=4.642089326307177
# time elapsed=1.09[min]|epoch=1|batch=[1551/1984]|c-loss=3.413344955444336|cf-loss=0.03704249151051044|s-loss=1.13555246591568|tv-loss=0.0|total loss=4.5859399128705265
# time elapsed=1.16[min]|epoch=1|batch=[1561/1984]|c-loss=3.476126194000244|cf-loss=0.04445799179375172|s-loss=1.1623533904552459|tv-loss=0.0|total loss=4.682937576249242
# time elapsed=1.22[min]|epoch=1|batch=[1571/1984]|c-loss=3.484347176551819|cf-loss=0.06247088136151433|s-loss=1.1881531596183776|tv-loss=0.0|total loss=4.734971217531711
# time elapsed=1.20[min]|epoch=1|batch=[1581/1984]|c-loss=3.460072875022888|cf-loss=0.04377928916364908|s-loss=1.1591040730476379|tv-loss=0.0|total loss=4.662956237234175
# time elapsed=1.19[min]|epoch=1|batch=[1591/1984]|c-loss=3.4454511642456054|cf-loss=0.04211148023605347|s-loss=1.1742143154144287|tv-loss=0.0|total loss=4.661776959896088
# time elapsed=1.21[min]|epoch=1|batch=[1601/1984]|c-loss=3.3701375246047975|cf-loss=0.025054193567484617|s-loss=1.206433153152466|tv-loss=0.0|total loss=4.601624871324748
# time elapsed=1.19[min]|epoch=1|batch=[1611/1984]|c-loss=3.3680969953536986|cf-loss=0.04089804030954838|s-loss=1.1509840488433838|tv-loss=0.0|total loss=4.559979084506631
# time elapsed=1.19[min]|epoch=1|batch=[1621/1984]|c-loss=3.395905041694641|cf-loss=0.048030875320546326|s-loss=1.1565413236618043|tv-loss=0.0|total loss=4.600477240676992
# time elapsed=1.16[min]|epoch=1|batch=[1631/1984]|c-loss=3.400654101371765|cf-loss=0.028303260169923305|s-loss=1.1388452053070068|tv-loss=0.0|total loss=4.5678025668486955
# time elapsed=1.16[min]|epoch=1|batch=[1641/1984]|c-loss=3.3955884695053102|cf-loss=0.03044873308390379|s-loss=1.1929109811782836|tv-loss=0.0|total loss=4.618948183767498
# time elapsed=1.16[min]|epoch=1|batch=[1651/1984]|c-loss=3.4301252603530883|cf-loss=0.08230480961501599|s-loss=1.1364468693733216|tv-loss=0.0|total loss=4.6488769393414255
# time elapsed=1.10[min]|epoch=1|batch=[1661/1984]|c-loss=3.451614809036255|cf-loss=0.0407200176268816|s-loss=1.2201490879058838|tv-loss=0.0|total loss=4.71248391456902
# time elapsed=1.22[min]|epoch=1|batch=[1671/1984]|c-loss=3.3553997755050657|cf-loss=0.0330932323820889|s-loss=1.2001820921897888|tv-loss=0.0|total loss=4.5886751000769435
# time elapsed=1.07[min]|epoch=1|batch=[1681/1984]|c-loss=3.471128296852112|cf-loss=0.04843454360961914|s-loss=1.1508923411369323|tv-loss=0.0|total loss=4.670455181598664
# time elapsed=1.22[min]|epoch=1|batch=[1691/1984]|c-loss=3.4608381509780886|cf-loss=0.049960554111748934|s-loss=1.1542222023010253|tv-loss=0.0|total loss=4.665020907390863
# time elapsed=1.15[min]|epoch=1|batch=[1701/1984]|c-loss=3.457601523399353|cf-loss=0.03452209550887346|s-loss=1.200075590610504|tv-loss=0.0|total loss=4.692199209518731
# time elapsed=1.12[min]|epoch=1|batch=[1711/1984]|c-loss=3.4568004846572875|cf-loss=0.028127316571772098|s-loss=1.22148756980896|tv-loss=0.0|total loss=4.70641537103802
# time elapsed=1.13[min]|epoch=1|batch=[1721/1984]|c-loss=3.445069742202759|cf-loss=0.05405349005013704|s-loss=1.1788386940956115|tv-loss=0.0|total loss=4.677961926348507
# time elapsed=1.19[min]|epoch=1|batch=[1731/1984]|c-loss=3.4266555309295654|cf-loss=0.04049448277801275|s-loss=1.1319843530654907|tv-loss=0.0|total loss=4.599134366773069
# time elapsed=1.20[min]|epoch=1|batch=[1741/1984]|c-loss=3.432933974266052|cf-loss=0.039654774405062196|s-loss=1.1830845832824708|tv-loss=0.0|total loss=4.655673331953585
# time elapsed=1.20[min]|epoch=1|batch=[1751/1984]|c-loss=3.3394370555877684|cf-loss=0.032215640600770715|s-loss=1.162038654088974|tv-loss=0.0|total loss=4.533691350277513
# time elapsed=1.26[min]|epoch=1|batch=[1761/1984]|c-loss=3.4450141191482544|cf-loss=0.07284950343891979|s-loss=1.1748370766639709|tv-loss=0.0|total loss=4.692700699251145
# time elapsed=1.15[min]|epoch=1|batch=[1771/1984]|c-loss=3.4153464317321776|cf-loss=0.04836533851921558|s-loss=1.1924267530441284|tv-loss=0.0|total loss=4.656138523295522
# time elapsed=1.26[min]|epoch=1|batch=[1781/1984]|c-loss=3.4284568071365356|cf-loss=0.05224130880087614|s-loss=1.1562187314033507|tv-loss=0.0|total loss=4.636916847340762
# time elapsed=1.20[min]|epoch=1|batch=[1791/1984]|c-loss=3.396366763114929|cf-loss=0.04890287332236767|s-loss=1.182088029384613|tv-loss=0.0|total loss=4.6273576658219095
# time elapsed=1.23[min]|epoch=1|batch=[1801/1984]|c-loss=3.382479190826416|cf-loss=0.04506317968480289|s-loss=1.1460234224796295|tv-loss=0.0|total loss=4.573565792990848
# time elapsed=1.12[min]|epoch=1|batch=[1811/1984]|c-loss=3.4656161069869995|cf-loss=0.05080460039898753|s-loss=1.1540091395378114|tv-loss=0.0|total loss=4.670429846923798
# time elapsed=1.24[min]|epoch=1|batch=[1821/1984]|c-loss=3.409670090675354|cf-loss=0.05312976324930787|s-loss=1.1664155721664429|tv-loss=0.0|total loss=4.629215426091105
# time elapsed=1.20[min]|epoch=1|batch=[1831/1984]|c-loss=3.464695692062378|cf-loss=0.05102228261530399|s-loss=1.1431812644004822|tv-loss=0.0|total loss=4.658899239078164
# time elapsed=1.22[min]|epoch=1|batch=[1841/1984]|c-loss=3.45927038192749|cf-loss=0.03284530779346824|s-loss=1.1059709787368774|tv-loss=0.0|total loss=4.598086668457836
# time elapsed=1.19[min]|epoch=1|batch=[1851/1984]|c-loss=3.3756930112838743|cf-loss=0.03789808354340494|s-loss=1.155181646347046|tv-loss=0.0|total loss=4.5687727411743255
# time elapsed=1.26[min]|epoch=1|batch=[1861/1984]|c-loss=3.479918432235718|cf-loss=0.05187477339059114|s-loss=1.1331280827522279|tv-loss=0.0|total loss=4.664921288378537
# time elapsed=1.18[min]|epoch=1|batch=[1871/1984]|c-loss=3.3567079067230225|cf-loss=0.04188534109853208|s-loss=1.171852707862854|tv-loss=0.0|total loss=4.570445955684408
# time elapsed=1.14[min]|epoch=1|batch=[1881/1984]|c-loss=3.3778866291046143|cf-loss=0.04243888380005956|s-loss=1.1544689893722535|tv-loss=0.0|total loss=4.574794502276927
# time elapsed=1.17[min]|epoch=1|batch=[1891/1984]|c-loss=3.5111618995666505|cf-loss=0.07765755765140056|s-loss=1.1694307565689086|tv-loss=0.0|total loss=4.75825021378696
# time elapsed=1.21[min]|epoch=1|batch=[1901/1984]|c-loss=3.3595887422561646|cf-loss=0.039517298247665165|s-loss=1.139168393611908|tv-loss=0.0|total loss=4.5382744341157375
# time elapsed=1.22[min]|epoch=1|batch=[1911/1984]|c-loss=3.437475800514221|cf-loss=0.050630005076527596|s-loss=1.143549120426178|tv-loss=0.0|total loss=4.631654926016926
# time elapsed=1.21[min]|epoch=1|batch=[1921/1984]|c-loss=3.4128281593322756|cf-loss=0.04807209027931094|s-loss=1.1527060508728026|tv-loss=0.0|total loss=4.613606300484389
# time elapsed=1.17[min]|epoch=1|batch=[1931/1984]|c-loss=3.420653200149536|cf-loss=0.05479414667934179|s-loss=1.1891185283660888|tv-loss=0.0|total loss=4.664565875194967
# time elapsed=1.16[min]|epoch=1|batch=[1941/1984]|c-loss=3.3953705787658692|cf-loss=0.07467554938048124|s-loss=1.1344122290611267|tv-loss=0.0|total loss=4.604458357207477
# time elapsed=1.15[min]|epoch=1|batch=[1951/1984]|c-loss=3.440773272514343|cf-loss=0.03916446790099144|s-loss=1.167246675491333|tv-loss=0.0|total loss=4.6471844159066675
# time elapsed=1.17[min]|epoch=1|batch=[1961/1984]|c-loss=3.443703866004944|cf-loss=0.04619986889883876|s-loss=1.1437158226966857|tv-loss=0.0|total loss=4.633619557600468
# time elapsed=1.16[min]|epoch=1|batch=[1971/1984]|c-loss=3.411359167098999|cf-loss=0.04489972945302725|s-loss=1.1188845038414001|tv-loss=0.0|total loss=4.575143400393427
# time elapsed=1.18[min]|epoch=1|batch=[1981/1984]|c-loss=3.3164549589157106|cf-loss=0.05927797751501203|s-loss=1.0752571761608123|tv-loss=0.0|total loss=4.450990112591535
# /usr/local/lib/python3.8/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([3, 64, 64])) that is different to the input size (torch.Size([4, 64, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
#   return F.mse_loss(input, target, reduction=self.reduction)